In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf

In [2]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [4]:
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

In [5]:
# will train a separate model for below two classes
pos_class_id = class_names.index("Pullover") 
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]


# Train model A with 8 classes

In [6]:
tf.random.set_seed(2024)
model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("models/transfer_learn/model_A/model_A.keras")

Epoch 1/20
1376/1376 [==============================] - 3s 2ms/step - loss: 1.1611 - accuracy: 0.6301 - val_loss: 0.6947 - val_accuracy: 0.7533
Epoch 2/20
1376/1376 [==============================] - 2s 2ms/step - loss: 0.6135 - accuracy: 0.7802 - val_loss: 0.5304 - val_accuracy: 0.8195
Epoch 3/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.5018 - accuracy: 0.8329 - val_loss: 0.4542 - val_accuracy: 0.8526
Epoch 4/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.4400 - accuracy: 0.8578 - val_loss: 0.4100 - val_accuracy: 0.8634
Epoch 5/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.4004 - accuracy: 0.8683 - val_loss: 0.3824 - val_accuracy: 0.8719
Epoch 6/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.3737 - accuracy: 0.8755 - val_loss: 0.3652 - val_accuracy: 0.8734
Epoch 7/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.3535 - accuracy: 0.8811 - val_loss: 0.3499 - val_accuracy:

# Train model B without transfer learning

In [7]:
tf.random.set_seed(2024)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))
model_B.evaluate(X_test_B, y_test_B)

Epoch 1/20
7/7 [==============================] - 1s 36ms/step - loss: 0.6320 - accuracy: 0.5900 - val_loss: 0.6231 - val_accuracy: 0.5905
Epoch 2/20
7/7 [==============================] - 0s 15ms/step - loss: 0.5943 - accuracy: 0.6500 - val_loss: 0.5884 - val_accuracy: 0.7181
Epoch 3/20
7/7 [==============================] - 0s 14ms/step - loss: 0.5614 - accuracy: 0.7700 - val_loss: 0.5649 - val_accuracy: 0.7913
Epoch 4/20
7/7 [==============================] - 0s 13ms/step - loss: 0.5393 - accuracy: 0.8650 - val_loss: 0.5450 - val_accuracy: 0.8457
Epoch 5/20
7/7 [==============================] - 0s 21ms/step - loss: 0.5196 - accuracy: 0.9150 - val_loss: 0.5302 - val_accuracy: 0.8625
Epoch 6/20
7/7 [==============================] - 0s 13ms/step - loss: 0.5045 - accuracy: 0.9250 - val_loss: 0.5156 - val_accuracy: 0.8853
Epoch 7/20
7/7 [==============================] - 0s 14ms/step - loss: 0.4900 - accuracy: 0.9500 - val_loss: 0.5020 - val_accuracy: 0.9041
Epoch 8/20
7/7 [===========

[0.3968988060951233, 0.9225000143051147]

# Train model B with transfer learning

## Clone model A

In [9]:
tf.random.set_seed(2024)

model_A = tf.keras.models.load_model('models/transfer_learn/model_A/model_A.keras')
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [10]:
model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1]) # all layers except last
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid")) # change activation func in last layer 

## Freeze layers

In [11]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [12]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

In [13]:
# train 4 epochs with frozen weights
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

# unfreeze weights
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

# recompile model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001) # rediced learning rate
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
# start training from last step
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 34ms/step - loss: 0.9463 - accuracy: 0.2650 - val_loss: 0.8734 - val_accuracy: 0.2918
Epoch 2/4
7/7 [==============================] - 0s 14ms/step - loss: 0.9297 - accuracy: 0.2550 - val_loss: 0.8494 - val_accuracy: 0.3066
Epoch 3/4
7/7 [==============================] - 0s 14ms/step - loss: 0.9040 - accuracy: 0.2550 - val_loss: 0.8355 - val_accuracy: 0.3304
Epoch 4/4
7/7 [==============================] - 0s 12ms/step - loss: 0.8870 - accuracy: 0.2650 - val_loss: 0.8163 - val_accuracy: 0.3462
Epoch 1/16
7/7 [==============================] - 1s 32ms/step - loss: 0.8633 - accuracy: 0.2850 - val_loss: 0.8045 - val_accuracy: 0.3759
Epoch 2/16
7/7 [==============================] - 0s 13ms/step - loss: 0.8503 - accuracy: 0.3150 - val_loss: 0.7914 - val_accuracy: 0.3937
Epoch 3/16
7/7 [==============================] - 0s 13ms/step - loss: 0.8347 - accuracy: 0.3350 - val_loss: 0.7797 - val_accuracy: 0.4085
Epoch 4/16
7/7 [===============